In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import os
import pickle

## read test file from disk

In [2]:
# data_test = pd.read_excel('test_set.xlsx')
data_test = pd.read_excel('test_set_rfs.xlsx')
   
for col in data_test.columns:
    data_test[col].replace(999,np.nan, inplace=True)

X_test = data_test.drop('ID',axis=1)

In [3]:
X_test

,Age,ER,PgR,HER2,TrippleNegative,ChemoGrade,Proliferation,HistologyType,LNStatus,TumourStage,...,original_glszm_SmallAreaHighGrayLevelEmphasis,original_glszm_SmallAreaLowGrayLevelEmphasis,original_glszm_ZoneEntropy,original_glszm_ZonePercentage,original_glszm_ZoneVariance,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength
0,36.000000,0,0.0,0.0,1.0,3.0,2.0,1.0,1,4,...,0.303965,0.303965,3.403702,0.001718,6.280592e+06,23.820975,0.010685,0.016075,1.900410e-04,0.010585
1,49.300000,1,1.0,1.0,0.0,2.0,1.0,1.0,0,2,...,0.311146,0.311146,3.449640,0.002814,2.899964e+06,107.424794,0.002625,0.076587,6.057930e-03,0.002319
2,53.200000,1,1.0,1.0,0.0,2.0,1.0,1.0,0,2,...,0.000010,0.000010,1.000000,0.000809,1.015056e+06,0.000000,1000000.000000,0.000000,0.000000e+00,0.000000
3,66.776181,1,1.0,0.0,0.0,2.0,1.0,2.0,0,4,...,0.761905,0.761905,1.220729,0.000427,1.022566e+08,4.141122,0.060412,0.000673,3.710000e-07,0.051226
4,53.683778,0,0.0,0.0,1.0,2.0,1.0,1.0,0,3,...,0.394504,0.394503,3.374972,0.005753,1.887473e+06,64.253074,0.004090,0.039622,1.443693e-03,0.003529
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,66.000000,0,0.0,0.0,1.0,2.0,1.0,2.0,1,4,...,0.543363,0.543363,2.324037,0.004338,3.224592e+06,24.328720,0.010442,0.013187,1.670920e-04,0.008646
76,36.900000,1,1.0,0.0,0.0,2.0,1.0,1.0,1,2,...,0.371461,0.371461,3.237445,0.006224,6.940156e+05,39.445275,0.006872,0.058431,3.150888e-03,0.006259
77,55.300000,0,0.0,0.0,1.0,2.0,1.0,1.0,0,3,...,0.138077,0.138076,3.418296,0.007430,1.370761e+05,37.228576,0.008853,0.139881,2.066187e-02,0.008096
78,48.100000,0,0.0,0.0,1.0,3.0,3.0,1.0,0,4,...,0.459564,0.459564,2.992289,0.001523,7.429380e+06,226.174716,0.001332,0.099413,1.102046e-02,0.001210


In [4]:
# Apply robust scaler to image data to scale them and remove outliers
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, KFold, RepeatedStratifiedKFold,RepeatedKFold, cross_val_score,cross_val_predict
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.compose import TransformedTargetRegressor

with open('reg_xgb.pkl','rb') as f:
    xgb = pickle.load(f)

y_pred = xgb.predict(X_test)

# mae_test=mean_absolute_error(y_true=y_test,y_pred=y_pred)
# print(f"XGB Mean absolute error is: {mae_test:.3f}")
y_pred=pd.DataFrame({'pred_RelapseFreeSurvival':y_pred})
data_pred=pd.merge(y_pred,data_test,left_index=True,right_index=True)
id = data_pred.pop('ID')
data_pred.insert(0,'ID',id)
data_pred

,ID,pred_RelapseFreeSurvival,Age,ER,PgR,HER2,TrippleNegative,ChemoGrade,Proliferation,HistologyType,...,original_glszm_SmallAreaHighGrayLevelEmphasis,original_glszm_SmallAreaLowGrayLevelEmphasis,original_glszm_ZoneEntropy,original_glszm_ZonePercentage,original_glszm_ZoneVariance,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength
0,TRG002539,59.981853,36.000000,0,0.0,0.0,1.0,3.0,2.0,1.0,...,0.303965,0.303965,3.403702,0.001718,6.280592e+06,23.820975,0.010685,0.016075,1.900410e-04,0.010585
1,TRG002806,49.590984,49.300000,1,1.0,1.0,0.0,2.0,1.0,1.0,...,0.311146,0.311146,3.449640,0.002814,2.899964e+06,107.424794,0.002625,0.076587,6.057930e-03,0.002319
2,TRG002857,75.939064,53.200000,1,1.0,1.0,0.0,2.0,1.0,1.0,...,0.000010,0.000010,1.000000,0.000809,1.015056e+06,0.000000,1000000.000000,0.000000,0.000000e+00,0.000000
3,TRG002631,53.738148,66.776181,1,1.0,0.0,0.0,2.0,1.0,2.0,...,0.761905,0.761905,1.220729,0.000427,1.022566e+08,4.141122,0.060412,0.000673,3.710000e-07,0.051226
4,TRG002709,52.898617,53.683778,0,0.0,0.0,1.0,2.0,1.0,1.0,...,0.394504,0.394503,3.374972,0.005753,1.887473e+06,64.253074,0.004090,0.039622,1.443693e-03,0.003529
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,TRG002271,78.813095,66.000000,0,0.0,0.0,1.0,2.0,1.0,2.0,...,0.543363,0.543363,2.324037,0.004338,3.224592e+06,24.328720,0.010442,0.013187,1.670920e-04,0.008646
76,TRG002886,61.416584,36.900000,1,1.0,0.0,0.0,2.0,1.0,1.0,...,0.371461,0.371461,3.237445,0.006224,6.940156e+05,39.445275,0.006872,0.058431,3.150888e-03,0.006259
77,TRG002797,52.434090,55.300000,0,0.0,0.0,1.0,2.0,1.0,1.0,...,0.138077,0.138076,3.418296,0.007430,1.370761e+05,37.228576,0.008853,0.139881,2.066187e-02,0.008096
78,TRG002883,51.200104,48.100000,0,0.0,0.0,1.0,3.0,3.0,1.0,...,0.459564,0.459564,2.992289,0.001523,7.429380e+06,226.174716,0.001332,0.099413,1.102046e-02,0.001210


In [5]:
## Export final file
data_pred.to_excel('FinalTestRFS.xlsx',index=False)